<a href="https://colab.research.google.com/github/Ayathussein09/Automated-Content-Generation-System/blob/main/Final_Project(Streamlit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers
!pip install streamlit Pillow gtts pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import login
login()

In [ ]:
from pyngrok import ngrok
!ngrok authtoken "2mbqfjmxAAFMAtTMflYYSXSDcgf_4dkf1n5NDNwLNbc57csM3"

# Terminate any active tunnels if necessary
# ngrok.kill()

# Reconnect ngrok after setting the authtoken
public_url = ngrok.connect(8501, "http")
print(public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
NgrokTunnel: "https://0579-34-16-185-216.ngrok-free.app" -> "http://localhost:8501"


In [ ]:

%%writefile app.py
# Streamlit UI
from io import BytesIO
import streamlit as st
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests
from transformers import pipeline
# import gradio as gr




class ImageDescription:
    def __init__(self):
        self.processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")
        self.model = LlavaNextForConditionalGeneration.from_pretrained(
                    "llava-hf/llava-v1.6-mistral-7b-hf",
                    torch_dtype=torch.float16,
                    device_map="auto",  # Automatically places layers on CPU/GPU to balance memory
                    low_cpu_mem_usage=True
                    )
        # self.model.to("cuda:0")


    def image_desc(self, img):
        image = img.resize((512, 512))  # Resize to 512x512
        conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Describe this image in Details"},
          {"type": "image"},
        ],
    },
]
        prompt = self.processor.apply_chat_template(conversation, add_generation_prompt=True)
        inputs = self.processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")
        output = self.model.generate(**inputs, max_new_tokens=100)
        return self.processor.decode(output[0], skip_special_tokens=True)




class EnhanceText:
    def __init__(self,system_messages=None):
        self.model_id = "meta-llama/Llama-3.2-3B-Instruct"
        self.pipe = pipeline(
          "text-generation",
          model=self.model_id,

          torch_dtype=torch.bfloat16,
          device_map="auto",
      )


    def generate_text(self, system_message, user_message, max_new_tokens=256):
        messages = [
            {"role": "system", "content": "Generate a prompt that is vivid, detailed, and imaginative, suitable for creating a complex, cinematic image in MidJourney. Focus on intricate visual descriptions, including lighting, mood, and atmosphere. Use cinematic framing, and emphasize color, texture, and scale. Make sure the prompt evokes a highly creative and fantastical image."},
            {"role": "user", "content": user_message},
        ]
        outputs = self.pipe(
            messages,
            max_new_tokens=max_new_tokens,
        )
        response = outputs[0]["generated_text"][-1]
        content_value = response['content']
        return content_value



    def enhance_text(self, content_value, max_new_tokens=256):
        messages = [
            {"role": "system", "content": "Enhance the quality of this prompt to be not too long"},
            {"role": "user", "content": content_value},
        ]
        outputs = self.pipe(
            messages,
            max_new_tokens=max_new_tokens,
        )
        response = outputs[0]["generated_text"][-1]
        return response['content']




image_description_obj = ImageDescription()
enhance_text_obj = EnhanceText()

  # Fixed variable name

# Streamlit App Layout
st.title("Image Description and Text Enhancement")

# Image Upload Section
st.header("Image Description")
uploaded_image = st.file_uploader("Upload an Image", type=["jpg", "png", "jpeg"])

if uploaded_image is not None:
    image = Image.open(uploaded_image)
    st.image(image, caption="Uploaded Image", use_column_width=True)

    if st.button("Generate Image Description"):
        with st.spinner('Generating description...'):
            description = image_description_obj.image_desc(image)
        st.write("**Image Description:**")
        st.write(description)


# Text Enhancement Section
st.header("Text Enhancement")
user_input_text = st.text_area("Enter your text for enhancement")

if st.button("Enhance Text"):
    if user_input_text:
        system_message = "Generate a prompt that is vivid, detailed, and imaginative, suitable for creating a complex, cinematic image in MidJourney. Focus on intricate visual descriptions, including lighting, mood, and atmosphere. Use cinematic framing, and emphasize color, texture, and scale. Make sure the prompt evokes a highly creative and fantastical image."
        with st.spinner('Enhancing text...'):
            generated_text = enhance_text_obj.generate_text(system_message, user_input_text)
            st.write("**Generated Text:**")
            st.write(generated_text)

        # Allow the user to edit the generated text and enhance further
        edited_text = st.text_area("Edit the generated text", value=generated_text)
        if st.button("Further Enhance Edited Text"):
            with st.spinner('Further enhancing text...'):
                enhanced_text = enhance_text_obj.enhance_text(edited_text)
            st.write("**Further Enhanced Text:**")
            st.write(enhanced_text)
    else:
        st.warning("Please enter text to enhance.")

Writing app.py


In [ ]:
from google.colab import files


In [ ]:
!streamlit run app.py --server.port 8501 --server.enableCORS false &


2024-10-04 06:28:59.139 
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.185.216:8501

2024-10-04 06:29:17.226672: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-04 06:29:17.252356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10